In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import json
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.preprocessing import LabelEncoder

# Load data
with open('/content/drive/MyDrive/cs-project dataset/intents.json', 'r') as file:
    data = json.load(file)

# Context management class
class ChatbotWithContext:
    def __init__(self):
        self.context = {}
        self.last_topic = None
        self.tokenizer = Tokenizer()
        self.model = None
        self.label_encoder = LabelEncoder()

    def train(self, data):
        patterns = []
        tags = []
        for intent in data['intents']:
            for pattern in intent['patterns']:
                patterns.append(pattern)
                tags.append(intent['tag'])

        # Preprocess
        self.tokenizer.fit_on_texts(patterns)
        X = self.tokenizer.texts_to_sequences(patterns)
        X = pad_sequences(X)
        y = self.label_encoder.fit_transform(tags)

        # Build model
        self.model = Sequential([
            Embedding(len(self.tokenizer.word_index) + 1, 128, input_length=X.shape[1]),
            LSTM(64),
            Dense(16, activation='relu'),
            Dropout(0.5),
            Dense(len(set(tags)), activation='softmax')
        ])

        self.model.compile(optimizer='adam',
                         loss='sparse_categorical_crossentropy',
                         metrics=['accuracy'])

        # Train
        self.model.fit(X, y, epochs=200, batch_size=32, validation_split=0.2)

    def predict_response(self, text):
        # Preprocess input
        X_test = self.tokenizer.texts_to_sequences([text])
        X_test = pad_sequences(X_test, maxlen=self.model.input_shape[1])

        # Get prediction
        prediction = self.model.predict(X_test)
        predicted_tag = self.label_encoder.inverse_transform([np.argmax(prediction)])

        # Update context
        current_topic = predicted_tag[0]
        self.context.update({
            'last_topic': self.last_topic,
            'current_topic': current_topic
        })
        self.last_topic = current_topic

        # Get response
        for intent in data['intents']:
            if intent['tag'] == predicted_tag[0]:
                response = np.random.choice(intent['responses'])
                return self.validate_response(text, response)

        return "I'm not sure how to respond to that."

    def validate_response(self, text, response):
        if len(response.split()) < 3:  # Too short
            return self.get_fallback_response()
        return response

    def get_fallback_response(self):
        return "Could you please provide more details about your question?"

# Usage
chatbot = ChatbotWithContext()
chatbot.train(data)

# Interactive loop
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chatbot.predict_response(user_input)
    print("Bot:", response)

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.1052 - loss: 3.1677 - val_accuracy: 0.0000e+00 - val_loss: 3.1641
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1930 - loss: 3.1135 - val_accuracy: 0.0000e+00 - val_loss: 3.1426
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1803 - loss: 3.0252 - val_accuracy: 0.0000e+00 - val_loss: 3.0988
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2008 - loss: 2.9345 - val_accuracy: 0.0000e+00 - val_loss: 3.0645
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1982 - loss: 2.8589 - val_accuracy: 0.0000e+00 - val_loss: 3.1167
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1978 - loss: 2.9258 - val_accuracy: 0.0000e+00 - val_loss: 3.2082
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.2689 - loss: 2.6786 - val_accuracy: 0.0000e+00 - val_loss: 3.2450
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.2863 - loss: 2.6215 - val_accuracy: 0

In [8]:
import pickle
from tensorflow.keras.models import load_model

# Save the model and tokenizer
chatbot.model.save('chatbot_model.keras')  # Save the model
with open('tokenizer.pickle', 'wb') as handle:  # Save the tokenizer
    pickle.dump(chatbot.tokenizer, handle)

# Load the model and tokenizer
chatbot.model = load_model('chatbot_model.keras')  # Load the model
with open('tokenizer.pickle', 'rb') as handle:  # Load the tokenizer
    chatbot.tokenizer = pickle.load(handle)

print("Model and tokenizer loaded successfully.")


Model and tokenizer loaded successfully.
